In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

api_key = os.getenv("OPEN_ROUTER_API_KEY")

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    model="openai/gpt-4o-mini",    
    model_info={
        "family": "openai",
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "structured_output": True
    },
)

In [ ]:
dsa_solver =  AssistantAgent(
    name="Complex_DSA_Solver",
    model_client = model_client,
    system_message="You are a helpful assistant that can solve complex DSA problems. give it in summary.",
    description="A complex DSA solver that can solve complex DSA problems.",
)

code_reviewer =  AssistantAgent(
    name="Code_Reviewer",
    model_client = model_client,
    system_message="You are a helpful assistant that can review code and provide feedback. give it in summary.",
    description="A code reviewer that can review code and provide feedback.",
)

code_editor =  AssistantAgent(
    name="Code_Editor",
    model_client = model_client,
    system_message="You are a helpful assistant that can edit code and provide feedback. give it in summary.",
    description="A code editor that can edit code and provide feedback.",
)

In [3]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage

team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=3
)

In [4]:
import asyncio
async def run_team():
    task = TextMessage(content="Write a simple python code for palindrom of words", source="user")
    result = await team.run(task=task)
    print(result.messages[-1].content)
await run_team()


The provided Python code effectively checks for palindromes, featuring a function that validates if a word reads the same forwards and backwards. Here are the key points:

### Key Features:
1. **Functionality**: The `is_palindrome` function checks for palindrome status.
2. **Case Normalization**: It converts the input to lowercase for case-insensitive comparison.
3. **String Reversal**: Uses Python's slicing to reverse the string and compare.
4. **User Input**: Allows dynamic user interaction for palindrome testing.

### Suggestions for Improvement:
- **Input Validation**: Add checks for non-string inputs to handle various data types gracefully.
- **Whitespace and Characters**: Strip out whitespace and punctuation to improve the function’s robustness.
- **Phrase Checking**: Expand functionality to check phrases or sentences for palindromic structure by ignoring spaces and punctuation.

Overall, the code is efficient and achieves its goal, but incorporating the suggestions could enhance

#### Since the Max turn is -> 1 the team will run only once from the DSA Solver and stops it after that

In [5]:
team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=1
)

In [6]:
import asyncio
async def run_team():
    task = TextMessage(content="Write a simple python code for palindrom of words", source="user")
    result = await team.run(task=task)
    print(result.messages[-1].content)
await run_team()


Here's a simple Python code snippet that checks if a given phrase (or word) is a palindrome, accounting for spaces and punctuation:

```python
import string

def is_palindrome(phrase):
    # Normalize the phrase: convert to lowercase and remove punctuation
    normalized_phrase = phrase.lower()
    normalized_phrase = normalized_phrase.translate(str.maketrans("", "", string.punctuation))
    normalized_phrase = normalized_phrase.replace(" ", "")
    
    # Check if the normalized phrase is equal to its reverse
    return normalized_phrase == normalized_phrase[::-1]

# Example usage
phrase = input("Enter a phrase or word: ")
if is_palindrome(phrase):
    print(f'"{phrase}" is a palindrome.')
else:
    print(f'"{phrase}" is not a palindrome.')
```

### Key Features of This Code:
- **Normalization**: Converts the input to lowercase and removes punctuation and spaces for accurate comparison.
- **Reversal Check**: Compares the normalized string with its reverse.
- **User Input**: Prompts th

In [7]:
team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=6
)

import asyncio
async def run_team():
    task = TextMessage(content="Write a simple python code for palindrom of words", source="user")
    result = await team.run(task=task)
    print(result)
await run_team()

messages=[TextMessage(id='9462641b-b937-421c-849a-a72e1d3e8be2', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 14, 17, 30, 44, 288268, tzinfo=datetime.timezone.utc), content='Write a simple python code for palindrom of words', type='TextMessage'), TextMessage(id='0724ec4b-7772-4aa7-b91e-ad3b51a25afa', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=536, completion_tokens=224), metadata={}, created_at=datetime.datetime(2026, 2, 14, 17, 30, 49, 792260, tzinfo=datetime.timezone.utc), content='Sure! Here\'s a simple Python code to check if a single word is a palindrome. This code will only consider the word without spaces or punctuation:\n\n```python\ndef is_palindrome(word):\n    # Remove spaces and convert to lowercase\n    word = word.replace(" ", "").lower()\n    # Check if the word is equal to its reverse\n    return word == word[::-1]\n\n# Example usage\nword = input("Enter a word: ")\nif is_palindrome(word):\n    print(f"{w

### There are differnet types of Termination one among them is:
### TextMention Termination, MaxMessage Termination

In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

my_termination = TextMentionTermination(text='TERMINATE') | MaxMessageTermination(max_messages=2)

team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    termination_condition=my_termination,
    max_turns=14
)

dsa_solver =  AssistantAgent(
    name="Complex_DSA_Solver",
    model_client = model_client,
    system_message="Your jobs is to solve complex DSA problems.",
)

code_reviewer =  AssistantAgent(
    name="Code_Reviewer",
    model_client = model_client,
    system_message="Yout job is to review the code. If the code is fine, Please say 'TERMINATE'",
)

code_editor =  AssistantAgent(
    name="Code_Editor",
    model_client = model_client,
    system_message="Your job is to edit the code. If the code Time complexity is more make it into O(n) or O(1) in the worst case as well",
)

async def run_team():
    task = TextMessage(content="Write a simple python code for addition of two numbers", source="user")
    result = await team.run(task=task)

    for each_agent in result.messages:
        print(f"{each_agent.source}: {each_agent.content}")
await run_team()

user: Write a simple python code for addition of tow numbers
Complex_DSA_Solver: Sure! Here’s a simple Python code to add two numbers:

```python
# Function to add two numbers
def add_numbers(num1, num2):
    return num1 + num2

# Input: Taking two numbers from the user
number1 = float(input("Enter the first number: "))
number2 = float(input("Enter the second number: "))

# Output: Display the result
result = add_numbers(number1, number2)
print("The sum of", number1, "and", number2, "is:", result)
```

You can run this code in any Python environment. It will prompt you to enter two numbers and then display their sum.
Code_Reviewer: TERMINATE


### the below code run_stream throws the output one be one as it is completed 

In [19]:
from autogen_agentchat.base import TaskResult
team2 = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    termination_condition=my_termination,
    max_turns=14
)
async for message in team2.run_stream(task="Write a simple hello world! code?"):
    print(type(message))
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message.source, message)

<class 'autogen_agentchat.messages.TextMessage'>
user id='706a0f95-474c-4467-a3cd-afa08650ee93' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 2, 14, 17, 59, 2, 660018, tzinfo=datetime.timezone.utc) content='Write a simple hello world! code?' type='TextMessage'
<class 'autogen_agentchat.messages.TextMessage'>
Complex_DSA_Solver id='544574d6-87ac-47dc-89d8-024e1a88be5f' source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=241, completion_tokens=224) metadata={} created_at=datetime.datetime(2026, 2, 14, 17, 59, 7, 679313, tzinfo=datetime.timezone.utc) content='Here’s a simple "Hello, World!" code snippet in several programming languages:\n\n### Python\n```python\nprint("Hello, World!")\n```\n\n### Java\n```java\npublic class HelloWorld {\n    public static void main(String[] args) {\n        System.out.println("Hello, World!");\n    }\n}\n```\n\n### JavaScript\n```javascript\nconsole.log("Hello, World!");\n```\n\n### C\n```c\n#include <stdio

## if i want to run the roundrobin for 2 times that means the termination max_messages = 7 (6 {teams} + 1 {user})

In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

my_termination = TextMentionTermination(text='TERMINATE') | MaxMessageTermination(max_messages=7)

team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    termination_condition=my_termination,
)

dsa_solver =  AssistantAgent(
    name="Complex_DSA_Solver",
    model_client = model_client,
    system_message="Your jobs is to solve complex DSA problems.",
)

code_reviewer =  AssistantAgent(
    name="Code_Reviewer",
    model_client = model_client,
    system_message="Yout job is to review the code. If the code is fine, Please say 'TERMINATE'",
)

code_editor =  AssistantAgent(
    name="Code_Editor",
    model_client = model_client,
    system_message="Your job is to edit the code. If the code Time complexity is more make it into O(n) or O(1) in the worst case as well",
)

async def run_team():
    task = TextMessage(content="Write a simple python code for addition of two numbers", source="user")
    result = await team.run(task=task)

    for each_agent in result.messages:
        print(f"{each_agent.source}: {each_agent.content}")
await run_team()

user: Write a simple python code for addition of tow numbers


In [21]:
state =  await team.save_state()
state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'Complex_DSA_Solver': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a simple hello world! code?',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'Here is a simple "Hello, World!" program in various programming languages:\n\n**Python:**\n```python\nprint("Hello, World!")\n```\n\n**Java:**\n```java\npublic class HelloWorld {\n    public static void main(String[] args) {\n        System.out.println("Hello, World!");\n    }\n}\n```\n\n**JavaScript:**\n```javascript\nconsole.log("Hello, World!");\n```\n\n**C:**\n```c\n#include <stdio.h>\n\nint main() {\n    printf("Hello, World!\\n");\n    return 0;\n}\n```\n\n**C++:**\n```cpp\n#include <iostream>\n\nint main() {\n    std::cout << "Hello, World!" << std::endl;\n    return 0;\n}\n```\n\n**Ruby:**\n```ruby\nputs "Hello,

In [34]:
agent1_agent_first =  AssistantAgent(
    name="agent1",
    model_client = model_client,
    system_message="add 1 to the number, first number is 0 and give the reuslt as ouput",
)

agent2_agent_second =  AssistantAgent(
    name="agent2",
    model_client = model_client,
    system_message="add 1 to the number, you got from the previous agent1 and give the reuslt as ouput",
)

agent3_agent_third =  AssistantAgent(
    name="agent3",
    model_client = model_client,
    system_message="add 1 to the number, you got from the previous agent3 and give the reuslt as ouput",
)

team3 = RoundRobinGroupChat(
    participants=[agent1_agent_first, agent2_agent_second, agent3_agent_third], max_turns=2
)

# the session is resumed from where it stopped

In [35]:
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent1) ----------
1
---------- TextMessage (agent2) ----------
2


TaskResult(messages=[TextMessage(id='68c064c5-b8d6-44ab-b314-cabc544af63c', source='agent1', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 17, 40, 268801, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='620e7f67-46fb-45c5-b69f-19ec366b1af9', source='agent2', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 17, 41, 399990, tzinfo=datetime.timezone.utc), content='2', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

In [36]:
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent3) ----------
3
---------- TextMessage (agent1) ----------
4


TaskResult(messages=[TextMessage(id='01baa804-d9e6-4812-90f8-938749545bb6', source='agent3', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 17, 42, 4361, tzinfo=datetime.timezone.utc), content='3', type='TextMessage'), TextMessage(id='9ddbd027-3fe2-446c-aa1a-fe4ea7afef2b', source='agent1', models_usage=RequestUsage(prompt_tokens=49, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 17, 42, 494818, tzinfo=datetime.timezone.utc), content='4', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

In [37]:
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent2) ----------
5
---------- TextMessage (agent3) ----------
6


TaskResult(messages=[TextMessage(id='7f1a262a-026a-495d-b204-8e410c40ded9', source='agent2', models_usage=RequestUsage(prompt_tokens=59, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 17, 43, 91885, tzinfo=datetime.timezone.utc), content='5', type='TextMessage'), TextMessage(id='19812db4-3ae0-4067-b008-ce0cc0578ef7', source='agent3', models_usage=RequestUsage(prompt_tokens=67, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 17, 43, 603338, tzinfo=datetime.timezone.utc), content='6', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

# after resting it

In [39]:
await team3.reset()
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent1) ----------
1
---------- TextMessage (agent2) ----------
2


TaskResult(messages=[TextMessage(id='cbed013c-1eae-4653-9498-d15892ed4f55', source='agent1', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 18, 38, 253736, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='bac94837-9b14-4ff7-9785-3d5320f51a84', source='agent2', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 14, 18, 18, 38, 852434, tzinfo=datetime.timezone.utc), content='2', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')